<a href="https://colab.research.google.com/github/shreyanshxt/brain-tumor-/blob/main/BRAIN_TUMOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nibabel


In [2]:
!pip install nilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 29.5 MB/s eta 0:00:00


In [ ]:
Original NIfTI (.nii) files
     ↓
[ANTs Registration] → aligned_suv.nii
     ↓
[Convert to NumPy] → aligned_suv.npy + cbf.npy
     ↓
[Stack or Train]   → [SUV, CBF] as input to your ViT


In [ ]:
#convert all the files in .suv folder to a .numpy format
import os
import numpy as np
suv_folder="/content/sample_data/(.suv)"
output_folder="/content/sample_data/output_folder"
os.makedirs(output_folder,exist_ok=True)
for filename in os.listdir(suv_folder):
  if filename.endswith(".nii"):
    img=nib.load(os.path.join(suv_folder,filename))
    data=img.get_fdata()
    np.save((os.path.join(output_folder,filename.replace(".nii",".npy"))),data)




In [ ]:
#convert all the files in .cbf folder to a .numpy format
import os
import numpy as np
suv_folder="/content/sample_data/(.cbf)"
output_folder="/content/sample_data/output_folder2"
os.makedirs(output_folder,exist_ok=True)
for filename in os.listdir(suv_folder):
  if filename.endswith(".nii"):
    img=nib.load(os.path.join(suv_folder,filename))
    data=img.get_fdata()
    np.save((os.path.join(output_folder,filename.replace(".nii",".npy"))),data)




In [7]:
!pip install antspyx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [9]:
#aligning the two suv and cbf
import nibabel as nib
import ants
import os
from os import path
import numpy as np
#loop to go through all teh files
suv_img_path="/content/sub-patient20_SUV_z.nii"
cbf_img_path="/content/sub-patient20_cbf.nii"


base, ext = os.path.splitext(suv_img_path)
cbf_img=ants.image_read(cbf_img_path)
suv_img=ants.image_read(suv_img_path)
suv_nii=nib.load(suv_img_path)
cbf_nii=nib.load(cbf_img_path)

img_cor_suv = ants.n4_bias_field_correction(suv_img)

img_cor_cbf = ants.n4_bias_field_correction(cbf_img)
# spacing
def spacing(affine):
  affine_data= affine.affine.astype(np.float64)
  return tuple(np.sqrt((affine_data[:3, :3] ** 2).sum(axis=0)))

#min max normalization
def minmax(img_ants):
  data=img_ants.numpy()
  normalized=(data-data.min())/(data.max()-data.min() + 1e-8)
  normalized = np.nan_to_num(normalized, nan=0.0, posinf=0.0, neginf=0.0)
  new_img = nib.Nifti1Image(
        normalized,
        affine=img_ants.affine,
        header=img_ants.header.copy()
    )
  return new_img # Return the new nibabel image


norm_suv=minmax(img_cor_suv)
norm_cbf=minmax(img_cor_cbf)

#covert npy->ants
ants_suv=ants.from_numpy(norm_suv.get_fdata())
ants_cbf=ants.from_numpy(norm_cbf.get_fdata())

#registration
reg=ants.registration(fixed=ants_suv,moving=ants_cbf,type_of_transform='SyN')
aligned_suv=reg['warpedmovout']
suv_reg=base+'_registered_'+ext
ants.image_write(aligned_suv,suv_reg)

TypeError: Nifti1Pair.__init__() got an unexpected keyword argument 'spacing'

In [ ]:
import ants
#loop to go through all teh files
suv_img_path="/content/sub-patient20_SUV_z.nii"
img=ants.image_read(suv_img_path)


In [ ]:

img.origin

(-270.0, -306.0, -72.0)

In [ ]:
img.shape

(181, 217, 181)

In [ ]:
import numpy as np
print("CBF Image Intensity Range:", np.min(cbf_img.numpy()),',', np.max(cbf_img.numpy()))
print("SUV Image Intensity Range:", np.min(suv_img.numpy()),',', np.max(suv_img.numpy()))


CBF Image Intensity Range: 0.0 , 280.5342
SUV Image Intensity Range: -2.7347765 , 3.4965055
